# Training "intermediate" approach

## Preparation

In [40]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os

In [41]:
project_path = Path().resolve().parent
input_path = project_path / "results" / "intermediate" / "training" / "intermediate"
#output_path = project_path / "results" / "MGE_files"

## Joined performance

In [47]:
thresholds=pd.read_csv(project_path / "results" / "intermediate" / "training" / "intermediate_thresholds.txt", header=None)
thresholds

,0
0,0.2 10 4 3
1,0.2 10 4 4
2,0.2 10 4 5
3,0.2 10 5 3
4,0.2 10 5 4
...,...
76,0.4 30 5 4
77,0.4 30 5 5
78,0.4 30 6 3
79,0.4 30 6 4


In [48]:
performance=pd.DataFrame(columns=['genome', 'score_threshold', 'n_threshold', 'initial_n_threshold', 'distance_threshold', 'nr_tool', 'nr_toolscheme', 'nr_scheme', 'avge_dispersion', 'coverage_denominator', 'coverage_numerator'])
#performance = pd.read_csv(input_path / str(i) / "joined_performance.csv", engine="python")
for i in thresholds[0]:
    perf = pd.read_csv(input_path / str(i) / "joined_performance.csv", engine="python", names=['genome', 'score_threshold', 'n_threshold', 'initial_n_threshold', 'distance_threshold', 'nr_tool', 'nr_toolscheme', 'nr_scheme', 'avge_dispersion', 'coverage_denominator', 'coverage_numerator'], header=None)
    performance=pd.concat([performance, perf])
performance['dispersion_weighted']=(performance.nr_toolscheme)*(pd.to_numeric(performance.avge_dispersion, errors='coerce'))
performance

,genome,score_threshold,n_threshold,initial_n_threshold,distance_threshold,nr_tool,nr_toolscheme,nr_scheme,avge_dispersion,coverage_denominator,coverage_numerator,dispersion_weighted
0,GCA_000820965.2,0.2,10,4,3,2,1,7,1.0,15,13,1.0
1,GCA_000963335.1,0.2,10,4,3,3,0,5,NaN,0,0,NaN
2,GCA_002860885.1,0.2,10,4,3,5,0,8,NaN,0,0,NaN
3,GCA_001682765.1,0.2,10,4,3,3,0,4,NaN,0,0,NaN
4,GCA_002289685.1,0.2,10,4,3,8,0,5,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
45,GCA_900092635.1,0.4,30,6,5,11,0,3,NaN,0,0,NaN
46,GCA_001754025.1,0.4,30,6,5,4,0,1,NaN,0,0,NaN
47,GCA_003186985.1,0.4,30,6,5,1,0,0,None,0,0,NaN
48,GCA_002138395.1,0.4,30,6,5,5,0,0,None,0,0,NaN


In [49]:
#perf_wide=performance.pivot(index='genome', columns='n_threshold', values=['nr_tool', 'nr_toolscheme', 'nr_scheme', 'avge_dispersion', 'coverage_denominator', 'coverage_numerator'])
#perf_wide




In [50]:
measures=performance.groupby(by=['score_threshold', 'n_threshold', 'initial_n_threshold', 'distance_threshold'], axis=0).sum()
measures=measures.loc[:, ['nr_tool', 'nr_toolscheme', 'nr_scheme', 'coverage_denominator', 'dispersion_weighted', 'coverage_numerator']]

In [51]:
measures['precision']=measures.nr_toolscheme/measures.nr_scheme
measures['recall']=measures.nr_toolscheme/measures.nr_tool
measures['coverage']=measures.coverage_numerator/measures.coverage_denominator
measures['dispersion']=measures.dispersion_weighted/measures.nr_toolscheme
measures['F-score']=2*measures.precision*measures.recall/(measures.precision+measures.recall)
measures.loc[:,['precision', 'recall', 'F-score', 'dispersion', 'coverage']]

precision  \
score_threshold n_threshold initial_n_threshold distance_threshold              
0.2             10          4                   3                    0.062162   
                                                4                    0.062857   
                                                5                    0.061584   
                            5                   3                    0.061224   
                                                4                    0.063325   
...                                                                       ...   
0.4             30          5                   4                    0.259259   
                                                5                    0.259259   
                            6                   3                    0.258065   
                                                4                    0.258065   
                                                5                    0.266667   

                                                                      recall  \
score_threshold n_threshold initial_n_threshold distance_threshold             
0.2             10          4                   3                   0.111111   
                                                4                   0.106280   
                                                5                   0.101449   
                            5                   3                   0.115942   
                                                4                   0.115942   
...                                                                      ...   
0.4             30          5                   4                   0.033816   
                                                5                   0.033816   
                            6                   3                   0.038647   
                                                4                   0.038647   
                                                5                   0.038647   

                                                                     F-score  \
score_threshold n_threshold initial_n_threshold distance_threshold             
0.2             10          4                   3                   0.079723   
                                                4                   0.078995   
                                                5                   0.076642   
                            5                   3                   0.080134   
                                                4                   0.081911   
...                                                                      ...   
0.4             30          5                   4                   0.059829   
                                                5                   0.059829   
                            6                   3                   0.067227   
                                                4                   0.067227   
                                                5                   0.067511   

                                                                    dispersion  \
score_threshold n_threshold initial_n_threshold distance_threshold               
0.2             10          4                   3                     1.391304   
                                                4                     1.363636   
                                                5                     1.380952   
                            5                   3                     1.375000   
                                                4                     1.375000   
...                                                                        ...   
0.4             30          5                   4                     1.000000   
                                                5                     1.000000   
                            6                   3                     1.000000   
                              